In [15]:
from dataclasses import dataclass
from pydantic import BaseModel

In [16]:
@dataclass
class PersonDataClass:
	name: str
	age: int = 0
	email: str | None = None

In [17]:
class PersonPydantic(BaseModel):
	name: str
	age: int = 0
	email: str | None = None

In [18]:
# Simple usage
peter_dataclass = PersonDataClass(name="Peter", age=35)
print(f'Using Dataclass: {peter_dataclass}')
peter_pydantic = PersonPydantic(name="Peter", age=35)
print(f'Using Pydantic: {peter_pydantic}')

Using Dataclass: PersonDataClass(name='Peter', age=35, email=None)
Using Pydantic: name='Peter' age=35 email=None


In [19]:
peter_dataclass = PersonDataClass("Peter", 35)
print(f'Using Dataclass: {peter_dataclass}')

# We Pydantic cannot use without positional arguments
peter_pydantic = PersonPydantic("Peter", 35)
print(f'Using Pydantic: {peter_pydantic}')

Using Dataclass: PersonDataClass(name='Peter', age=35, email=None)


TypeError: BaseModel.__init__() takes 1 positional argument but 3 were given

In [ ]:
# Initialize with dictionary
martin_data = {"name": "Martin", "age": 35, "email": "martin@doublephd.com"}
martin_dataclass = PersonDataClass(**martin_data)
print(f'Using Dataclass: {martin_dataclass}')

martin_pydantic = PersonPydantic(**martin_data)
print(f'Using Pydantic: {martin_pydantic}')

# The same
martin_pydantic = PersonPydantic.model_validate(martin_data)
print(f'Using Pydantic: {martin_pydantic}')

Using Dataclass: PersonDataClass(name='Martin', age=35, email='martin@doublephd.com')
Using Pydantic: name='Martin' age=35 email='martin@doublephd.com'
Using Pydantic: name='Martin' age=35 email='martin@doublephd.com'


In [ ]:
# Dump to dictionary
martin_dict_data = martin_pydantic.model_dump()
print(f'Dict: {martin_dict_data}')
print(f'Dict Name:{martin_dict_data["name"]}')

martin_dict_data = martin_pydantic.model_dump()
print(f'Dict email: {martin_dict_data}')

# Dump to dictionary without email
print('\nDump to dictionary without email')
martin_dict_data = martin_pydantic.model_dump(exclude={"email"})
print(f'Dict without email: {martin_dict_data}')
print(f'Dict Name: {martin_dict_data.get("email", "Not found")}')

Dict: {'name': 'Martin', 'age': 35, 'email': 'martin@doublephd.com'}
Dict Name:Martin
Dict email: {'name': 'Martin', 'age': 35, 'email': 'martin@doublephd.com'}
Dict Email:martin@doublephd.com

Dump to dictionary without email
Dict without email: {'name': 'Martin', 'age': 35}
Dict Name: Not found


In [ ]:
### JSON operations
martin_json_data = martin_pydantic.model_dump_json(exclude={"email"})
print(f'JSON: {martin_json_data}')
# save json file
with open("martin.json", "w") as f:
    f.write(martin_json_data)

# Load from json file
with open("martin.json", "r") as f:
    martin_json_read_data = f.read()
    martin_pydantic = PersonPydantic.model_validate_json(martin_json_read_data)
    print(f'Read from json file: {martin_pydantic}')

JSON: {"name":"Martin","age":35}
Read from json file: name='Martin' age=35 email=None


In [ ]:
### Validation

john_data = {"age": "John", "name": 35}
john_dataclass = PersonDataClass(**john_data)
print(f'Using Dataclass: {john_data}')
# No Validation!

john_pydantic = PersonPydantic(**john_data)
print(f'Using Pydantic: {john_pydantic}')

Using Dataclass: {'age': 'John', 'name': 35}


ValidationError: 2 validation errors for PersonPydantic
name
  Input should be a valid string [type=string_type, input_value=35, input_type=int]
    For further information visit https://errors.pydantic.dev/2.6/v/string_type
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='John', input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/int_parsing

In [ ]:
## Coercions

jacob = {"name": "Jacob", "age": "44" }
jacob_dataclass = PersonDataClass(**jacob)
print(f'Using Dataclass: {jacob_dataclass}')
print(f'Type of age: {type(jacob_dataclass.age)}')

jacob_pydantic = PersonPydantic(**jacob)
print(f'Using Pydantic: {jacob_pydantic}')
print(f'Type of age: {type(jacob_pydantic.age)}')

Using Dataclass: PersonDataClass(name='Jacob', age='44', email=None)
Type of age: <class 'str'>
Using Pydantic: name='Jacob' age=44 email=None
Type of age: <class 'int'>


In [ ]:

# Validation with Pydantic
from pydantic import EmailStr, Field, PositiveInt
# EmailStr requires pip install pydantic[email]

class PersonWithValidation(BaseModel):
    name: str = Field(min_length=3, max_length=50)
    age: PositiveInt
    email: EmailStr | None = None

peter = PersonWithValidation(name="Peter", age=35, email="peter@my.com")

In [20]:
from pydantic import ValidationError


negative_age_data = {"name": "Peter", "age": -35}
try:
    peter = PersonWithValidation(**negative_age_data)
except ValidationError as e:
    print(e)

1 validation error for PersonWithValidation
age
  Input should be greater than 0 [type=greater_than, input_value=-35, input_type=int]
    For further information visit https://errors.pydantic.dev/2.6/v/greater_than


In [21]:
to_short_name_data = {"name": "Pe", "age": 35}
try:
    peter = PersonWithValidation(**to_short_name_data)
except ValidationError as e:
    print(e)

1 validation error for PersonWithValidation
name
  String should have at least 3 characters [type=string_too_short, input_value='Pe', input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/string_too_short


In [22]:
bad_email_data = {"name": "Peter", "age": 35, "email": "peter"}
try:
    peter = PersonWithValidation(**bad_email_data)
except ValidationError as e:
    print(e)

1 validation error for PersonWithValidation
email
  value is not a valid email address: The email address is not valid. It must have exactly one @-sign. [type=value_error, input_value='peter', input_type=str]


In [27]:
class NameStartWithP(BaseModel):
    name: str = Field(min_length=3, max_length=50, pattern="^P.*$")

class PersonWithValidation(BaseModel):
    name: NameStartWithP
    age: PositiveInt
    email: EmailStr | None = None

peter = PersonWithValidation(name="John", age=35)
try:
    peter = PersonWithValidation(name="John", age=35)
except ValidationError as e:
    print(e)

ValidationError: 1 validation error for PersonWithValidation
name
  Input should be a valid dictionary or instance of NameStartWithP [type=model_type, input_value='John', input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/model_type

In [28]:
from pydantic import field_validator, model_validator


class UserModel(BaseModel):
    name: str
    surname: str

    @field_validator('name')
    @classmethod
    def name_must_start_with_capital_letter(cls, v: str) -> str:
        if not v[0].isupper():
            raise ValueError('name must start with a capital letter')
        return v
    
    @model_validator(mode='after')
    def name_and_surname_must_be_different(self) -> 'UserModel':
        if self.name == self.surname:
            raise ValueError('name and surname must be different')
        return self

user_model_correct = UserModel(name="Peter", surname="Smith")
print(user_model_correct)
user_model_incorrect = UserModel(name="Peter", surname="Peter")
try:
    user_model_incorrect = UserModel(name="Peter", surname="Peter")
except ValidationError as e:
    print(e)

name='Peter' surname='Smith'


ValidationError: 1 validation error for UserModel
  Value error, name and surname must be different [type=value_error, input_value={'name': 'Peter', 'surname': 'Peter'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/value_error

In [26]:
from pydantic import computed_field


class BoundingBox(BaseModel):
    width: float
    height: float

    @computed_field
    def area(self) -> float:
        return self.width * self.height

box = BoundingBox(width=10, height=20)
print(box)
print(box.area)

width=10.0 height=20.0 area=200.0
200.0
